In [109]:
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import pandas as pd
import requests
import datetime

In [5]:
binance_api_key = 'E90xR7UnO2oeP3lrgC50zkqUuAqyyXs4uSlAGcqIVO9n90ELrFI4gfVI2HGGxxCj'
binance_api_secret='Inls2xYl9FqPi0zlIrucfxG81drx7w6Pv6SzYFxMNoAcwDjlCKug7zUM9zB3lF9h'

In [6]:
client = Client(binance_api_key,binance_api_secret)

In [7]:
def get_market_cap(quote="USDT"):
    
    resp = requests.get("https://www.binance.com/bapi/asset/v2/public/asset-service/product/get-products")
    
    market_cap=pd.DataFrame(resp.json()['data'])
    market_cap=market_cap[market_cap['q']==quote]
    
    market_cap=market_cap[['an','qn','s','b','q','c','cs']]
    market_cap['c']=market_cap['c'].astype(float)
    
    market_cap.columns=['Long name','Quote Name','Ticker','Short Name','Quote Short Name','Close','Supply']
    market_cap['Market Cap']=market_cap['Close']*market_cap['Supply']
    
    market_cap=market_cap.sort_values(by='Market Cap',ascending=False)
    

    return market_cap

In [145]:

def get_price(ticker_list,date=str(datetime.date.today())):
    

    price=pd.DataFrame()

    for ticker in ticker_list:

        temp=pd.DataFrame()
        
        numeric_columns =  ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Asset Volume', 
                        'Number of Trades', 'TB Base Volume', 'TB Quote Volume', 'Ignore']
        
        data = pd.DataFrame(client.get_historical_klines(ticker, Client.KLINE_INTERVAL_1DAY, date),columns=numeric_columns)
        data['Close Time']=pd.to_datetime(data['Close Time'], unit='ms').dt.date
        data=data.set_index('Close Time')

        temp[ticker]=data['Close'].astype(float)
        
        #price=pd.concat([price,temp[ticker]],axis=1)
        price=pd.concat([price,temp[ticker]],axis=1)
        
    return price

In [ ]:
"""Credit to https://github.com/stevebelgium/binance_spot_EURO/blob/master/binancewalleteuro.py"""

def get_account_balances(binance_api_key,binance_api_secret):
	
	# Create a new client object to interact with the Binance API
	client = Client(binance_api_key, binance_api_secret)

	# Retrieve the balances of all coins in the user's Binance account
	account_balances = client.get_user_asset()

	# Get the current price of all tickers from the Binance API
	ticker_info = client.get_all_tickers()

	# Create a dictionary of tickers and their corresponding prices
	ticker_prices = {ticker['symbol']: float(ticker['price']) for ticker in ticker_info}

	# Calculate the USDT value of each coin in the user's account
	coin_values = []
	for coin_balance in account_balances:
		# Get the coin symbol and the free and locked balance of each coin
		coin_symbol = coin_balance['asset']
		unlocked_balance = float(coin_balance['free'])
		locked_balance = float(coin_balance['locked'])

		# If the coin is USDT and the total balance is greater than 1, add it to the list of coins with their EURO values
		if coin_symbol == 'USDT' and unlocked_balance + locked_balance > 1:
			# Calculate EURO value using EURUSDT coin pair
			coin_values.append(('USDT', (unlocked_balance + locked_balance)/ticker_prices.get('EURUSDT')))
		# Otherwise, check if the coin has a USDT trading pair or a BTC trading pair
		elif unlocked_balance + locked_balance > 0.0:
			# Check if the coin has a USDT trading pair
			if (any(coin_symbol + 'USDT' in i for i in ticker_prices)):
				# If it does, calculate its USDT value and add it to the list of coins with their EURO values
				ticker_symbol = coin_symbol + 'USDT'
				ticker_price = ticker_prices.get(ticker_symbol)
				coin_usdt_value = (unlocked_balance + locked_balance) * ticker_price
				if coin_usdt_value > 1:
					# Calculate EURO value using EURUSDT coin pair
					coin_values.append((coin_symbol, coin_usdt_value/ticker_prices.get('EURUSDT')))
			# If the coin does not have a USDT trading pair, check if it has a BTC trading pair
			elif (any(coin_symbol + 'BTC' in i for i in ticker_prices)):
				# If it does, calculate its USDT value and add it to the list of coins with their EURO values
				ticker_symbol = coin_symbol + 'BTC'
				ticker_price = ticker_prices.get(ticker_symbol)
				coin_usdt_value = (unlocked_balance + locked_balance) * ticker_price * ticker_prices.get('BTCUSDT')
				if coin_usdt_value > 1:
					# Calculate EURO value using EURUSDT coin pair
					coin_values.append((coin_symbol, coin_usdt_value/ticker_prices.get('EURUSDT')))

	# Sort the list of coins and their EURO values by EURO value in descending order
	coin_values.sort(key=lambda x: x[1], reverse=True)
    
	# Return the list of coins and their EURO values
	return pd.DataFrame(coin_values,columns=['Ticker','Amount in €'])

